# CosFace, ArcFace

Косинусно-угловые функции потерь: 
* CosFace (http://openaccess.thecvf.com/content_cvpr_2018/papers/Wang_CosFace_Large_Margin_CVPR_2018_paper.pdf, CVPR)
* ArcFace (https://arxiv.org/abs/1801.07698).

Наиболее актуальным нейросетевым методом для распознавания лиц (да и идентификации любых других сущностей) являются косинусно-угловые методы. Как и другие популярные подходы к обучению метрик (triplet и contrastive loss), они обладают хорошей обобщающей способностью и обучают непосредственно вектор-представление, однако не требуют сложных метоов составления пар или триплетов, являясь практически такими же простыми в использовании и обучении, как обычная классификация с использованием перекрёстной энтропии и softmax (softmax loss).

Softmax loss можно представить следующим образом:

$$ - \frac{1}{N} \sum_{i=1}^{N} \log{\frac{e^{W_{y_i}^{T} x_i + b_{y_i}} }{\sum_{j=1}^{N} e^{W_{y_i}^{T} x_j + b_j}  }}$$


Здесь N - размер батча, n - количество классов, $x_i \in R^d$ вектор-эмбеддинг, соответствующий классу $y_i$, $W \in R^{d\times n}$ - веса между последним и предпоследним полносвязными слоями, $W_j \in R^d$ - j-ый столбец матрицы $W$, который представляет основанный на эмбеддинге линейный классификатор для соответствующего класса, $b_i \in R^n$ - смещение.

Подходы данной группы предполагают, что $W_j$ представляют центры классов в угловом пространстве, и представляют $W_j x$ как $||W|| * ||x|| * cos(\theta_j)$, где $\theta_j$ - угол между $W_j$ и $x$. И CosFace, и ArcFace нормализуют как $x$, так и $W_j$, а также фиксируют смещение, делая его равным нулю, что практически не отражается на результате, после чего softmax loss начинает зависеть только от $cos(\theta_j)$ :

$$ - \frac{1}{N} \sum_{i=1}^{N} \log{ \frac{e^{\cos(\theta_{y_i})} }{e^{\cos(\theta_{y_i})} + \sum_{j=1,j\neq y_i}^{N} e^{\cos(\theta_j)} } } $$


Они также вводят понятие scale, отвечающее за масштабирование получающихся эмбеддингов: $||x|| = scale$, а также угловой зазор margin, отвечающий за дискриминативность получающихся эмбеддингов (помогает “расталкивать” соответствующие классам группы получившихся эмбеддингов друг от друга в угловом пространстве), чего не хватает подходу с обычной классификацией. Для CosFace итоговая функция потерь выглядит следующим образом:

$$ - \frac{1}{N} \sum_{i=1}^{N} \log{\frac{e^{scale \times (\cos(\theta_{y_i})-margin)} }{e^{scale \times (\cos(\theta_{y_i})-margin)} + \sum_{j=1,j \neq y_i}^{N} e^{scale \times \cos(\theta_j)} } }$$


ArcFace же непосредственно добавляет margin к получившемуся углу:

$$ - \frac{1}{N} \sum_{i=1}^{N} \log{\frac{e^{ scale \times \cos(\theta_{y_i}+margin)} }{e^{scale \times \cos(\theta_{y_i}+margin)} + \sum_{j=1,j \neq y_i}^{N} e^{scale \times \cos(\theta_j)} } }$$

При этом вычислить угол в случае использования нормализации крайне просто:

$$ \cos(\theta_{j}) = \frac{W_j \circ x}{||W_j|| \times ||x||} = W_j \circ x$$

# Код
CosFace и ArcFace реализованы с использованием keras (tensorflow backend), в качестве игрушечного примера взян MNIST. Имплементировано как слой, так как иначе Keras API не позволяет передать веса функции потерь. Предсказание класса по получаемому сетью эмбеддингу осуществляется с помощью KNN и эмбеддингов, полученных из тренировочных данных.

In [1]:
import tensorflow as tf
import numpy as np
from keras.datasets import mnist

from keras.layers import Layer, Input, Activation, BatchNormalization, Conv2D, Dense, Flatten, Lambda
from keras.initializers import glorot_uniform
from keras.models import Model
import math

from keras.optimizers import Adam, SGD, RMSprop, Adagrad
from keras.callbacks import ModelCheckpoint
import os

from sklearn.neighbors import KNeighborsClassifier

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [2]:
class DenseCosFaceLoss(Layer):
    def __init__(self, output_dim, margin=30, scale=0.35, **kwargs):
        self.output_dim = output_dim
        self.margin = margin
        self.scale = scale
        super(DenseCosFaceLoss, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)
        super(DenseCosFaceLoss, self).build(input_shape)

    def call(self, x):
        self.kernel = tf.nn.l2_normalize(self.kernel, 0, 1e-10)
        x = tf.nn.l2_normalize(x, 1, 1e-10)
        self.cos_t = tf.matmul(x, self.kernel)
        return self.cos_t

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def loss(self, labels, _features):
        labels = tf.cast(tf.squeeze(labels), tf.uint8)
        labels = tf.one_hot(labels, self.output_dim, on_value=1.0, off_value=0.0, axis=-1, dtype=tf.float32)
        
        cosine = tf.clip_by_value(self.cos_t, -1, 1) - self.margin * labels
        
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=self.scale * cosine))

    def get_config(self):
        config = {'output_dim': self.output_dim,
                  'm': self.margin,
                  'scale': self.scale}
        base_config = super(DenseCosFaceLoss, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class DenseArcFaceLoss(Layer):
    def __init__(self, output_dim, margin=64, scale=0.5, **kwargs):
        # margin should be in [0, pi/2)
        self.output_dim = output_dim
        self.margin = margin
        self.scale = scale
        super(DenseArcFaceLoss, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel',
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)
        super(DenseArcFaceLoss, self).build(input_shape)

    def call(self, x):
        self.kernel = tf.nn.l2_normalize(self.kernel, 0, 1e-10)
        x = tf.nn.l2_normalize(x, 1, 1e-10)
        self.cos_t = tf.matmul(x, self.kernel)
        return self.cos_t

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def loss(self, labels, _features):
        labels = tf.cast(tf.squeeze(labels), tf.uint8)
        labels = tf.one_hot(labels, self.output_dim, on_value=1.0, off_value=0.0, axis=-1, dtype=tf.float32)
        
        cos_m = math.cos(self.margin)
        sin_m = math.sin(self.margin)

        # get cos(t + m) without uning arccos(cos(t)). arccos is bad computationally
        sin_t = tf.sqrt(1. - tf.square(self.cos_t))
        cos_margined = self.scale * (cos_m * self.cos_t - sin_m * sin_t)

        # for "cos(t1 + m) > cos(t2)" we want "t1 + m" to be in [0, pi]: otherwise m only makes "cos(t1 + m)" bigger,
        # and it's not what is margin for. So when cos(t) < cos(pi - m) -> cos(t + m) > pi, and in this case
        # we will just use cosface with some adaptive cosface margin built from arcface margin via m * sin(m)
        threshold = math.cos(math.pi - self.margin)
        switch_cosface = tf.to_float(self.cos_t >= threshold) * (self.cos_t - sin_m * self.margin)

        arc = self.cos_t * (1. - labels) + tf.where(switch_cosface > 0., cos_margined, switch_cosface) * labels
        return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=self.scale * arc))
    
    def get_config(self):
        config = {'output_dim': self.output_dim,
                  'm': self.margin,
                  'scale': self.scale}
        base_config = super(DenseArcFaceLoss, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    
def get_mnist_model(input_shape=(28, 28, 1), n_classes=10, mode='cosface', train=False):
    img = Input(input_shape)

    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same', name='conv_1', kernel_initializer=glorot_uniform())(img)
    x = BatchNormalization(axis=3, name='bn_1')(x)
    x = Activation('relu')(x)

    x = Conv2D(64, (3, 3), strides=(2, 2), padding='same', name='conv_2', kernel_initializer=glorot_uniform())(x)
    x = BatchNormalization(axis=3, name='bn_2')(x)
    x = Activation('relu')(x)

    x = Conv2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv_3', kernel_initializer=glorot_uniform())(x)
    x = BatchNormalization(axis=3, name='bn_3')(x)
    x = Activation('relu')(x)

    x = Flatten()(x)
    x = Dense(64, activation='relu', kernel_initializer=glorot_uniform())(x)
    
    if train:
        if mode == 'cosface':
            x = DenseCosFaceLoss(n_classes, margin=30, scale=0.35)(x)
        elif mode == 'arcface':
            x = DenseArcFaceLoss(n_classes, margin=64, scale=0.5)(x)
    else:
        x = Lambda(lambda x:  tf.nn.l2_normalize(x, 1, 1e-10))(x)
        

    model = Model(inputs=img, outputs=x, name='mnist_model')
    return model

In [3]:
def train_cosangular(learning_rate=0.01, batch_size=50, epochs=20, mode='cosface'):
    if not os.path.isdir('data/training'):
        os.makedirs('data/training')
        
    model = get_mnist_model(mode=mode, train=True)
    model.summary()
    model.compile(optimizer=Adam(learning_rate), loss=model.layers[-1].loss)
    
    model.fit(np.expand_dims(x_train, axis=-1), y_train, batch_size=batch_size, epochs=epochs,
              callbacks=[ModelCheckpoint(filepath=os.path.join('data', 'training', 'checkpoint-{epoch:02d}.h5'),
                                         save_weights_only=True)])
    
train_cosangular(learning_rate=0.0001, batch_size=300, epochs=50, mode='arcface')

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv_1 (Conv2D)              (None, 14, 14, 32)        320       
_________________________________________________________________
bn_1 (BatchNormalization)    (None, 14, 14, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 7, 7, 64)          18496     
_________________________________________________________________
bn_2 (BatchNormalization)    (None, 7, 7, 64)          256       
_________________________________________________________________
acti

In [5]:
def validate_cosangular():
    model = get_mnist_model(train=False)
    model.load_weights('data/trained_weights.h5', by_name=True, skip_mismatch=True)
    
    train_embeddings = model.predict(np.expand_dims(x_train, axis=-1))
    val_embeddings = model.predict(np.expand_dims(x_test, axis=-1))
    
    KNN = KNeighborsClassifier(n_neighbors=50, metric='sqeuclidean', weights='distance')
    KNN.fit(train_embeddings, y_train)
    
    pred = KNN.predict(val_embeddings)
    
    acc = sum(y_test == pred) / len(pred)
    print('accuracy: ', acc)
    
validate_cosangular()

accuracy:  0.9621
